In [ ]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np
import nltk
import sklearn
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
pip install newspaper3k

# Crawling News from TOI Website

In [ ]:
pip install datefinder

In [ ]:
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)
print(r.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd"><html class="mweb"><head ><meta content="The Times of India" property="og:site_name"><meta content="website" property="og:type"><meta content="556964827" property="fb:admins"><meta content="117787264903013" property="fb:app_id"><meta content="26781952138" property="fb:pages"><meta content="IE=edge" http-equiv="X-UA-Compatible"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta content="ZgFICIedNvVZl5pV9EfAUeenwta9vBY0Za_GgmV4zuw" name="google-site-verification"><META content="b1abb36a8d5c19c9" name="y_key"><meta value="summary_large_image" name="twitter:card"><meta value="@timesofindia" name="twitter:site"><meta name="keywords" content="world news,latest world news,world breaking news,global news,international news,news of the world,us news,top world news,uk news,live world news,times of india world news"><title>World News Headlines, Latest International News, World Breaking 

Using BeautifulSoup to extract the tag tree structure and creating a list of anchor tags containing links belonging to articles of class 'w_img'.


In [ ]:
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.findAll('a', attrs = {'class':'w_img'})
print(table) 

[<a class="w_img" href="/world/middle-east/as-coronavirus-cases-rise-uae-adjusts-to-a-new-normal-in-pandemic/articleshow/75423087.cms" pg="-world/middle-east#Story_View-1-geturl" title="As coronavirus cases rise, UAE adjusts to a new normal in pandemic"><img alt="As coronavirus cases rise, UAE adjusts to a new normal in pandemic" data-src="https://static.toiimg.com/photo/75423307.cms" pg="-world/middle-east#Story_View-1-geturl" src="https://static.toiimg.com/photo/34824568.cms"/></a>, <a class="w_img" href="/videos/international/covid-19-mayor-entertains-residents-after-imposing-curfew/videoshow/75398612.cms" pg="-videos#Video_View-1-geturl" title="Covid-19: Mayor entertains residents after imposing curfew"><img alt="Covid-19: Mayor entertains residents after imposing curfew" data-src="https://static.toiimg.com/thumb/msid-75398612,width-255,resizemode-4,imgsize-167996/75398612.jpg" pg="-videos#Video_View-1-geturl" src="https://static.toiimg.com/photo/34824568.cms"/><i class="iconvideo"

Correcting links missing https

In [ ]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

Downloading each article and extracting title, text and keywords and summary, storing it in list of dictionaries.

In [ ]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

Creating dataframe out of above created list.

In [ ]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,"As coronavirus cases rise, UAE adjusts to a ne...",A gate system made by Guard ME that conducts t...,He believes the new safety measures will find ...,"[temperature, rise, normal, coronavirus, dubai..."
1,Covid-19: Mayor entertains residents after imp...,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","[way, entertains, dont, tshirt, mayor, true, i..."
2,UK PM Boris Johnson returns to work after reco...,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...","Apr 27, 2020, 08:40AM ISTSource: Times NowBrit...","[worst, street, coronavirus, times, recovering..."
3,"1 officer dead, 1 wounded in Louisiana shooting","Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...","Apr 27, 2020, 08:28AM ISTSource: APPolice say ...","[standoff, law, shot, officers, paul, wounded,..."
4,Covid-19: Lockdown reveals fresh and clean air...,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...","Apr 22, 2020, 04:03PM ISTSource: APIndia's ext...","[unprecedented, stay, transport, reveals, aver..."


# Useign RandomForest Model for predicting virality

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
def clean_cols(data):
    """Clean the column names by stripping and lowercase."""
    clean_col_map = {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=clean_col_map)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
    """Easy Train Test Split call."""
    return train_test_split(X, Y, test_size=test_size, random_state=R)

The data has following features:
Number of Attributes: 61 (58 predictive attributes, 2 non-predictive, 1 goal field)

Attribute Information:
0. url: URL of the article (non-predictive)
1. timedelta: Days between the article publication and the dataset acquisition (non-predictive)
2. n_tokens_title: Number of words in the title
3. n_tokens_content: Number of words in the content
4. n_unique_tokens: Rate of unique words in the content
5. n_non_stop_words: Rate of non-stop words in the content
6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content
7. num_hrefs: Number of links
8. num_self_hrefs: Number of links to other articles published by Mashable
9. num_imgs: Number of images
10. num_videos: Number of videos
11. average_token_length: Average length of the words in the content
12. num_keywords: Number of keywords in the metadata
13. data_channel_is_lifestyle: Is data channel 'Lifestyle'?
14. data_channel_is_entertainment: Is data channel 'Entertainment'?
15. data_channel_is_bus: Is data channel 'Business'?
16. data_channel_is_socmed: Is data channel 'Social Media'?
17. data_channel_is_tech: Is data channel 'Tech'?
18. data_channel_is_world: Is data channel 'World'?
19. kw_min_min: Worst keyword (min. shares)
20. kw_max_min: Worst keyword (max. shares)
21. kw_avg_min: Worst keyword (avg. shares)
22. kw_min_max: Best keyword (min. shares)
23. kw_max_max: Best keyword (max. shares)
24. kw_avg_max: Best keyword (avg. shares)
25. kw_min_avg: Avg. keyword (min. shares)
26. kw_max_avg: Avg. keyword (max. shares)
27. kw_avg_avg: Avg. keyword (avg. shares)
28. self_reference_min_shares: Min. shares of referenced articles in Mashable
29. self_reference_max_shares: Max. shares of referenced articles in Mashable
30. self_reference_avg_sharess: Avg. shares of referenced articles in Mashable
31. weekday_is_monday: Was the article published on a Monday?
32. weekday_is_tuesday: Was the article published on a Tuesday?
33. weekday_is_wednesday: Was the article published on a Wednesday?
34. weekday_is_thursday: Was the article published on a Thursday?
35. weekday_is_friday: Was the article published on a Friday?
36. weekday_is_saturday: Was the article published on a Saturday?
37. weekday_is_sunday: Was the article published on a Sunday?
38. is_weekend: Was the article published on the weekend?
39. LDA_00: Closeness to LDA topic 0
40. LDA_01: Closeness to LDA topic 1
41. LDA_02: Closeness to LDA topic 2
42. LDA_03: Closeness to LDA topic 3
43. LDA_04: Closeness to LDA topic 4
44. global_subjectivity: Text subjectivity
45. global_sentiment_polarity: Text sentiment polarity
46. global_rate_positive_words: Rate of positive words in the content
47. global_rate_negative_words: Rate of negative words in the content
48. rate_positive_words: Rate of positive words among non-neutral tokens
49. rate_negative_words: Rate of negative words among non-neutral tokens
50. avg_positive_polarity: Avg. polarity of positive words
51. min_positive_polarity: Min. polarity of positive words
52. max_positive_polarity: Max. polarity of positive words
53. avg_negative_polarity: Avg. polarity of negative words
54. min_negative_polarity: Min. polarity of negative words
55. max_negative_polarity: Max. polarity of negative words
56. title_subjectivity: Title subjectivity
57. title_sentiment_polarity: Title polarity
58. abs_title_subjectivity: Absolute subjectivity level
59. abs_title_sentiment_polarity: Absolute polarity level
60. shares: Number of shares (target)

Out of these I dropped some of these features for training as I cannot calculate these for the crawled data.
I am using no. of shares as target parameter for measuring virality.


In [ ]:
full_data = clean_cols(pd.read_csv("/content/drive/My Drive/OnlineNewsPopularity/OnlineNewsPopularity.csv"))
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta', 'lda_00','lda_01','lda_02','lda_03','lda_04','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta', 'num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']

Training the classifier.

In [ ]:
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

Predicting the test set.

In [ ]:
rf_res = pd.DataFrame(clf.predict(x_train),list(y_train))

Printing Actual shares vs predicted shares.

In [ ]:
rf_res.reset_index(level=0, inplace=True)
rf_res_df = rf_res.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
rf_res_df.head()

,Actual shares,Predicted shares
0,16100,11247.11
1,508,882.22
2,1300,1864.97
3,3100,3627.22
4,6900,5415.02


MSE of the trained model.

In [ ]:
print(sklearn.metrics.mean_squared_error(rf_res_df['Actual shares'], rf_res_df['Predicted shares'], multioutput='uniform_average'))

20255201.234858133


# Preparing our crawled data in format of training data.

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

Return rate of unique words in a list of words.

In [ ]:
def rate_unique(words):
    words=tokenize(words)
    no_order = list(set(words))
    rate_unique=len(no_order)/len(words)
    return rate_unique

Returns Rate of non-stop words in a list of words.

In [ ]:
def rate_nonstop(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    rate_nonstop=len(filtered_sentence)/len(words)
    no_order = list(set(filtered_sentence))
    rate_unique_nonstop=len(no_order)/len(words)
    return rate_nonstop,rate_unique_nonstop

Return the average length of tokenised words.

In [ ]:
def avg_token(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [ ]:
from textblob import TextBlob

Return Date of articles.

In [ ]:
import datefinder
import datetime  
from datetime import date 
def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article)))>0:
        date=str(list(datefinder.find_dates(article))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday"

return tokenized form of text.

In [ ]:
def tokenize(text):
    text=text
    return word_tokenize(text)

For returning the polarity of the word.

In [ ]:
pos_words=[]
neg_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            pos_words.append(i)
        if polarity<0:
            neg_words.append(i)
    return pos_words,neg_words

Return rate of positive, negative, average positive and negative polarities in the article.

In [ ]:
def rates(words):
    words=polar(words)
    pos=words[0]
    neg=words[1]
    all_words=words
    global_rate_positive_words=(len(pos)/len(all_words))/100
    global_rate_negative_words=(len(neg)/len(all_words))/100
    pol_pos=[]
    pol_neg=[]
    for i in pos:
        analysis=TextBlob(i)
        pol_pos.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in neg:
        analysis2=TextBlob(j)
        pol_neg.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(pol_pos)
    max_positive_polarity=max(pol_pos)
    min_negative_polarity=min(pol_neg)
    max_negative_polarity=max(pol_neg)
    avg_positive_polarity=np.average(pol_pos)
    avg_negative_polarity=np.average(pol_neg)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity


Generzting the data.

In [ ]:
df2=[]
for i in news:
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=rate_unique(article.text)
    pred_info['n_non_stop_words']=rate_nonstop(article.text)[0]
    pred_info['n_non_stop_unique_tokens']=rate_nonstop(article.text)[1]
    pred_info['num_hrefs']=article.html.count("https://timesofindia.indiatimes.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(i)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(i)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(i)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(i)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(i)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(i)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(i)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]    
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    df2.append(pred_info)

In [ ]:
pred_df=pd.DataFrame(df2)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()

,text,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity
0,A gate system made by Guard ME that conducts t...,13,971,0.511843,0.662204,0.443872,231,13,0,4.613800,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0.390260,0.045572,0.135,0.16,0.257239,0.033333,0.5,-0.177902,-0.6,-0.05,0.552273,0.143182
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",8,190,0.594737,0.589474,0.431579,185,9,0,3.726316,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.643750,0.300000,1.110,0.64,0.268221,0.033333,0.6,-0.177902,-0.6,-0.05,0.000000,0.000000
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",11,97,0.773196,0.711340,0.597938,185,9,0,4.505155,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.415625,-0.153125,1.325,0.67,0.281137,0.033333,0.6,-0.210421,-1.0,-0.05,0.000000,0.000000
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",9,153,0.614379,0.692810,0.483660,185,9,0,4.326797,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0.085714,0.028571,1.370,0.77,0.276128,0.033333,0.6,-0.218807,-1.0,-0.05,0.400000,-0.200000
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",10,124,0.701613,0.766129,0.604839,185,9,0,4.443548,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.534242,0.108939,1.470,0.83,0.274941,0.033333,0.6,-0.220064,-1.0,-0.05,0.600000,0.333333


# Results for our crawled data.

In [ ]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,A gate system made by Guard ME that conducts t...,19838.73
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",23898.88
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",25067.02
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",17732.44
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",22819.96
5,"Apr 22, 2020, 03:58PM IST\n\nSource: AP\n\nIra...",22729.98
6,"Apr 22, 2020, 04:04PM IST\n\nSource: AP\n\nAn ...",19043.25
7,"Apr 22, 2020, 03:57PM IST\n\nSource: AP\n\nFou...",18740.37
8,"Apr 21, 2020, 09:25AM IST\n\nSource: Times Now...",26028.45
9,"Apr 21, 2020, 03:38PM IST\n\nSource: AP\n\nA s...",22576.90
